In [1]:
from gpt2.modeling_gpt2 import GPT2LMHeadModel
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers import BertTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
from datetime import datetime
import os
import numpy as np
import logging
import random
from torch.utils.tensorboard import SummaryWriter

In [14]:
def seed_everything(seed: int = 42):
    """Util to make training reproducible"""
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if os.getenv("CUBLAS_WORKSPACE_CONFIG") is not None:
        torch.use_deterministic_algorithms(True)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def worker_init(worked_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def set_logger(path):

    logger = logging.getLogger()
    handler = logging.FileHandler(path + "/train_log.txt")
    logger.setLevel(level=logging.INFO)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "%(asctime)s - %(filename)s - %(funcName)s - %(lineno)s - %(levelname)s\n%(message)s",
        "%Y-%m-%d %H:%M:%S",
    )
    handler.setFormatter(formatter)
    console = logging.StreamHandler()
    console.setFormatter(formatter)
    logger.addHandler(handler)
    logger.addHandler(console)


class Chinese_Medical_DS(Dataset):
    def __init__(self, path, tokenizer, max_len=1024):
        self.path = path
        sentence = []
        self.private_positions_list = []
        
        self.total_private_tokens = 0
        self.total_tokens = 0
        self.private_tokens_per_sentence = []
        
        with open(self.path, 'r', encoding='utf-8') as f:
            for line in f:
                sen_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(line.strip()))
                full_sen = []
                
                full_sen.append(tokenizer.convert_tokens_to_ids('[MASK]'))
                full_sen.extend(sen_ids)
                full_sen.append(tokenizer.convert_tokens_to_ids('[CLS]'))
                private_positions = [i for i, token_id in enumerate(full_sen) if tokenizer.decode([token_id]).isdigit()]
                self.private_positions_list.append(private_positions)
                if len(full_sen) <= max_len:
                    sentence.append(full_sen)
                
                num_private_tokens = len(private_positions)
                self.total_private_tokens += num_private_tokens
                self.total_tokens += len(full_sen)
                self.private_tokens_per_sentence.append(num_private_tokens)
                
        self.data = sentence
        
        # 计算平均每句话中的隐私token数
        average_private_tokens_per_sentence = sum(self.private_tokens_per_sentence) / len(self.private_tokens_per_sentence)

        # 计算总的隐私token占总token的比例
        private_token_ratio = self.total_private_tokens / self.total_tokens

        # 计算隐私token个数的均值和方差
        private_token_mean = np.mean(self.private_tokens_per_sentence)
        private_token_variance = np.var(self.private_tokens_per_sentence)

        # 打印统计信息
        print(f"Average private tokens per sentence: {average_private_tokens_per_sentence:.2f}")
        print(f"Private token ratio: {private_token_ratio:.4f}")
        print(f"Private token count mean: {private_token_mean:.2f}")
        print(f"Private token count variance: {private_token_variance:.2f}")
        
    # need to overload
    def __len__(self):
        return len(self.data)

    # need to overload
    def __getitem__(self, idx):
        input = self.data[idx]
        target = input
        private_positions = self.private_positions_list[idx]
        return input, target, private_positions
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, save_path, patience=2, verbose=True, delta=0):
        """
        Args:
            save_path : 模型保存文件夹
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.save_path = save_path
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(self.save_path + 'best_model')
        
        # path = os.path.join(self.save_path, "best_network.pth")
        # torch.save(model.state_dict(), path)  # 这里会存储迄今最优模型的参数
        self.val_loss_min = val_loss

In [15]:
seed_everything()
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
pretrain_model_path = "..\\..\\Raw_GPT2\\"
output_dir = "model\\"

epochs = 50
warmup_steps = 1000
lr = 1e-5
gradient_accumulation = 18
max_grad_norm = 1.0
log_step = 10000
set_logger(output_dir)
logger = logging.getLogger(__name__)


In [16]:
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(pretrain_model_path)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info('using device:{}'.format(device))
model.train()
model.to(device)
logger.info(model)

GPT2LMHeadModel::init
config =  GPT2Config {
  "_name_or_path": "..\\..\\Raw_GPT2\\",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 320
    }
  },
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.24.0",
  "use_cache

2023-04-01 14:36:42 - 945538542.py - <module> - 4 - INFO
using device:cuda
2023-04-01 14:36:42 - 945538542.py - <module> - 4 - INFO
using device:cuda
2023-04-01 14:36:42 - 945538542.py - <module> - 4 - INFO
using device:cuda
2023-04-01 14:36:43 - 945538542.py - <module> - 7 - INFO
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
   

In [17]:
train_dataset = Chinese_Medical_DS("..\\..\\Data\\tiny_train.txt", tokenizer)
train_dataloader = DataLoader(dataset=train_dataset, worker_init_fn=worker_init)
valid_dataset = Chinese_Medical_DS("..\\..\\Data\\tiny_valid.txt", tokenizer)
valid_dataloader = DataLoader(dataset=valid_dataset, worker_init_fn=worker_init)
logger.info("len(train_dataloader), len(valid_dataloader) = {}, {}".format(len(train_dataloader), len(valid_dataloader)))

Average private tokens per sentence: 1.22
Private token ratio: 0.0055
Private token count mean: 1.22
Private token count variance: 4.91


2023-04-01 14:37:13 - 3687209076.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500
2023-04-01 14:37:13 - 3687209076.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500
2023-04-01 14:37:13 - 3687209076.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500


Average private tokens per sentence: 1.22
Private token ratio: 0.0055
Private token count mean: 1.22
Private token count variance: 5.27


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                          num_training_steps=len(train_dataloader))
tb_path = output_dir + "/tb"
if not os.path.exists(tb_path):
    os.mkdir(tb_path)
writer = SummaryWriter(tb_path)

In [10]:
train_num_log = len(train_dataloader) * np.log(len(train_dataloader))
delta = 1.0 / train_num_log if 1.0 / train_num_log < 1e-5 else 1e-5
epsilon = 0.5
sigma = 0.810546875
epsilon, delta, sigma

(0.5, 1e-05, 0.810546875)

In [13]:
running_loss = 0
early_stopping = EarlyStopping(output_dir)
train_step_per_epoch = len(train_dataloader)
valid_step_per_epoch = len(valid_dataloader)
for epoch in range(epochs):
    logger.info('epoch {}'.format(epoch + 1))
    now = datetime.now()
    logger.info('time: {}'.format(now))
    model.train()
    train_pbar = tqdm(train_dataloader)
    all_train_loss = 0.0
    train_pbar.set_description('epoch-' + str(epoch + 1))
    
    for step, (input, label, private_positions) in enumerate(train_pbar):
        input_ids = torch.tensor(label).long().to(device)
        label_ids = torch.tensor(input).long().to(device)
        private_positions = torch.tensor(private_positions, dtype=torch.long).to(device)

        #  forward pass
        outputs = model(input_ids=input_ids, labels=label_ids, private_positions=private_positions, sigma=sigma)
        loss, logits = outputs[:2]
        
        if gradient_accumulation > 1:
            loss = loss / gradient_accumulation
            
        #  loss backward
        # if fp16:
        #     with amp.scale_loss(loss, optimizer) as scaled_loss:
        #         scaled_loss.backward()
        #         torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
        # else:
        #     loss.backward()
        #     torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        loss.backward()
        loss = loss.detach()
        all_train_loss += loss
        
        writer.add_scalar('loss/train_step_loss', scalar_value=loss * gradient_accumulation, global_step=epoch * train_step_per_epoch+step)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        #  optimizer step
        if (step + 1) % gradient_accumulation == 0:
            running_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        if (step + 1) % log_step == 0:
            logger.info('now time: {}:{}. Step {} of epoch {}, loss {}'.format(
                datetime.now().hour,
                datetime.now().minute,
                (step + 1) // gradient_accumulation,
                epoch + 1,
                running_loss / log_step))
            running_loss = 0
        
        train_pbar.set_postfix({'loss': '{:.7f}'.format(loss*gradient_accumulation)})
        
    logger.info('train step = {}'.format(step))
    all_train_loss = all_train_loss / (step + 1)

    writer.add_scalar('loss/train_epoch_loss', scalar_value=all_train_loss * gradient_accumulation, global_step=epoch + 1)
    logger.info('saving model for epoch {}'.format(epoch + 1))
    if not os.path.exists(output_dir + 'model_epoch{}'.format(epoch + 1)):
        os.mkdir(output_dir + 'model_epoch{}'.format(epoch + 1))
    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(output_dir + 'model_epoch{}'.format(epoch + 1))

    logger.info('epoch {} finished, train loss = {:.10f}'.format(epoch + 1, all_train_loss * gradient_accumulation))

    then = datetime.now()
    logger.info('time: {}'.format(then))
    logger.info('time for one epoch: {}'.format(then - now))
    
    logger.info('start validate')
    model.eval()
    all_valid_loss = 0.0
    valid_pbar = tqdm(valid_dataloader)
    valid_pbar.set_description('valid ' + str(epoch + 1))
    for step, (input, label) in enumerate(valid_pbar):
        input_ids = torch.tensor(label).long().to(device)
        label_ids = torch.tensor(input).long().to(device)

        #  forward pass
        outputs = model(input_ids=input_ids, labels=label_ids)
        loss = outputs[0].detach()
        writer.add_scalar('loss/valid_step_loss', scalar_value=loss, global_step=epoch * valid_step_per_epoch + step)
        all_valid_loss += loss
        valid_pbar.set_postfix({'loss': '{:.7f}'.format(loss)})
    
    logger.info('valid step = {}'.format(step))
    all_valid_loss = all_valid_loss / (step + 1)
    writer.add_scalar('loss/valid_epoch_loss', scalar_value=all_valid_loss, global_step=epoch+1)
    logger.info('valid finished, valid loss = {:.10f}'.format(all_valid_loss))
    early_stopping(all_valid_loss, model)
    if early_stopping.early_stop:
        logger.info("Early stopping")
        break

writer.close()    

logger.info('training finished')
if not os.path.exists(output_dir + 'final_model'):
    os.mkdir(output_dir + 'final_model')
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir + 'final_model')

2023-04-01 14:31:40 - 2840535088.py - <module> - 6 - INFO
epoch 1
2023-04-01 14:31:40 - 2840535088.py - <module> - 6 - INFO
epoch 1
2023-04-01 14:31:40 - 2840535088.py - <module> - 8 - INFO
time: 2023-04-01 14:31:40.233154
2023-04-01 14:31:40 - 2840535088.py - <module> - 8 - INFO
time: 2023-04-01 14:31:40.233154
epoch-1:   0%|          | 0/9000 [00:00<?, ?it/s, loss=4.0664477]

input_ids =  tensor([ 103, 2140, 2140,  671, 4684, 4717,  833, 3221, 1538, 3694, 1036, 8043,
        2769, 2157, 2140, 2140,  738, 3221, 3300,  673,  702, 3299,  749, 8024,
         702, 2094, 2798, 8399, 8024,  860, 7028, 2798, 1282, 1061, 3165, 8024,
        5307, 2382, 2476, 2458, 1673, 2349,  679, 5052, 3221, 4717, 1286, 6230,
        6820, 3221, 4635, 1921, 8024, 2769, 6574, 4542,  800, 3221, 1538, 3694,
        1036, 8024, 6821, 1008, 1408, 8043, 6716, 3332, 4765, 2207, 8024, 1928,
        1741,  856,  754, 3633, 2382, 8024, 1928, 1184,  510, 1400, 2520, 4764,
        8024, 3359, 6956, 2398, 1439, 2793, 1928,  511, 7568, 4764,  510, 4649,
        5502, 2160, 3351,  511, 7755, 7977, 2382, 5862, 1400,  754, 2399, 7977,
        8024, 1139, 4280, 2454, 1400,  684, 2382, 7231,  855,  511, 1928, 1355,
        5301, 6763, 5445, 6772, 2208,  511, 1184, 1727, 1394, 2879, 3241, 8024,
        7553, 3359,  704, 5296, 1377, 3300, 5018,  753, 1727, 7305,  511, 1724,
        5501, 4764, 8024, 4

epoch-1:   0%|          | 1/9000 [00:00<32:41,  4.59it/s, loss=2.8775215]

input_ids =  tensor([ 103, 2111, 2094, 2793, 3425,  860, 1355, 4142, 1355, 4173, 2582, 3416,
        1394, 4415, 7650, 7608, 8043, 2769, 2157, 4638, 2111, 2094, 3221, 4511,
        2140, 2140, 8024,  124, 2259, 8024, 1157, 2458, 1993, 8024, 1624, 2094,
        4706, 3300, 4157, 4578, 8024, 1355, 4385, 8024,  845, 3300, 6768, 2544,
        4638, 1495, 1644, 8024, 1369, 1912, 8024, 3300, 4157, 1355, 4173,  738,
        3766, 5125, 4868, 8024, 6435, 7309, 8038, 2111, 2094, 2793, 3425,  860,
        1355, 4142, 1355, 4173, 2582, 3416, 1394, 4415, 7650, 7608,  511, 1355,
        4385, 2793, 3425,  860, 4142, 1218, 2553, 6206, 1350, 3198, 5314, 2111,
        2094, 3780, 4545, 8024, 1369, 1912, 3189, 2382, 7650, 7608, 1377,  809,
        1914, 1391,  671,  763, 5922, 5831, 1469, 3717, 3362, 8024, 7370,  749,
        3926, 3909,  679, 1173, 4080,  722, 1912, 8024, 6821, 5102, 7608, 4289,
        4638, 5852, 1075, 5162,  738, 3221, 3683, 6772,  705, 2168, 4638, 8024,
        3300, 1221,  754, 2

epoch-1:   0%|          | 5/9000 [00:00<14:41, 10.21it/s, loss=2.7803459]

...

epoch-1:   0%|          | 35/9000 [00:02<09:47, 15.27it/s, loss=2.9400237]

input_ids =  tensor([ 103, 1278, 7368, 2582,  720, 3780, 3255, 1213,  856,  678, 6821,  702,
        4567,  136, 2207, 2111, 2094, 2347, 5307,  758, 2259,  749, 8024, 2697,
        6230, 3300,  763, 2460, 2382, 8024,  679, 1008, 3633, 2382, 4638, 2111,
        2094, 8024, 2600, 3221, 4717, 6230, 8024, 1353, 2418, 6826, 7162, 8024,
        3221,  679, 3221, 3255, 1213,  856,  678, 8043, 4385, 1762, 2682, 4761,
        6887, 8024, 1278, 7368, 2582,  720, 3780, 3255, 1213,  856,  678, 6821,
         702, 4567,  136, 3301, 1351, 8024, 3418, 2945,  872, 4638, 1360, 6835,
        8024, 1036, 4997, 3255, 1213,  856,  678, 7444, 6206, 3300,  757, 4685,
        6981, 1394, 6817, 4500, 1278, 2110,  510, 4852,  833,  510, 3136, 2193,
        1469, 5466,  689, 4294, 6378, 5023, 2974, 3177, 8024, 2902, 2399, 7977,
        1920, 2207, 1469, 3255, 1213,  856,  678, 4638,  698, 7028, 4923, 2428,
        2190, 2642, 5442, 2141, 3177, 4294, 6378, 8024,  886, 1071, 6809, 1168,
        2226, 1377, 5543, 7

epoch-1:   0%|          | 36/9000 [00:02<11:30, 12.99it/s, loss=2.9400237]


input_ids =  tensor([ 103,  122, 2259,  130,  702, 3299, 1920,  912, 3198, 1526, 3221, 6768,
        2544,  912, 4908, 2582,  720, 1215, 8043, 6435, 7309,  117, 2207, 1957,
         122, 2259,  130,  702, 3299,  117, 1762, 1920,  912, 3198, 1962, 1526,
         117, 3221, 6768, 2544,  912, 4908,  117, 6421, 2582, 3416, 3780, 4545,
        1450,  136, 1961, 2398, 2382, 1391,  763, 5922, 5831, 7313, 7392, 1391,
        8024, 6820, 3300, 6007,  510, 3717, 7659,  510, 3739,  510, 7672, 1928,
         510, 4156, 2372, 7824,  510, 2207, 5101, 3739, 8024,  671, 5663, 7650,
        7608,  738, 2218, 6821,  763, 8024, 7370,  749, 1914, 1600, 3717, 8024,
        6820, 3300, 1166, 4638, 3175, 3791, 1408, 8043, 6468, 6468, 8013,  872,
        1962, 8024,  671, 2259, 1914, 2140, 2140,  912, 4908, 3680, 3613, 1920,
         912, 6963, 1526, 8024, 1377, 5543, 3221, 7270, 4574, 4555,  749,  117,
         679,  833, 6432, 6413,  117, 3680, 3613, 2861, 5107, 5107, 2218, 1526,
         117, 1377, 5543, 3

KeyboardInterrupt: 